# Scalarization testing


### ASF

Now there is standard Achievement Scalarizing Function implemented, so let's use it.

In [1]:
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
import logging

In [2]:
logger = logging.getLogger('pyomo.opt')
logger.setLevel(logging.ERROR)

In [3]:
revenue, carbon, deadwood, ha = init_boreal()
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('glpk')

In [4]:
ind = 100
x = np.dstack((new_normalize(nan_to_bau(revenue[:ind]).values),
               new_normalize(nan_to_bau(carbon[:ind]).values),
               new_normalize(nan_to_bau(deadwood[:ind]).values),
               new_normalize(nan_to_bau(ha[:ind]).values)))
X = np.dstack((nan_to_bau(revenue[:ind]).values,
               nan_to_bau(carbon[:ind]).values,
               nan_to_bau(deadwood[:ind]).values,
               nan_to_bau(ha[:ind]).values))

In [5]:
%%time
ref = np.array((ide[0], 0, 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [6]:
%%time
ref = np.array((0, ide[1], 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model,X), axis=0))

In [7]:
%%time
ref = np.array((0, 0, ide[2], 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

### Timing

The ASF function seems to work properly so no problem. Let's see how much time does it take when running in full speed.

In [8]:
%%time
n_revenue = nan_to_bau(revenue)
n_carbon = nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha= nan_to_bau(ha)
x_full = np.dstack((new_normalize(n_revenue.values),
                    new_normalize(n_carbon.values),
                    new_normalize(n_deadwood.values),
                    new_normalize(n_ha.values)))
X_full = np.dstack((n_revenue.values,
                    n_carbon.values,
                    n_deadwood.values,
                    n_ha.values))

In [9]:
from interactiveBoreal import ReferenceFrame as rf
kehys = rf()

In [10]:
kyle1 = np.array((1858.0, 58.69, 2.651, 0.27))
ref1 = kyle1 * 68700

In [11]:
nor_ide = kehys.normalize_ref(ide)
nor_nad = kehys.normalize_ref(nad)
nor_ref = kehys.normalize_ref(ref1)

In [12]:
%%time
asf_fullx = ASF(nor_ide, nor_nad, nor_ref, x_full)
opt.solve(asf_fullx.model)
print(np.sum(values_to_list(asf_fullx.model,X_full), axis=0))

In [27]:
['{:11.2f}'.format(r) for r in np.sum(values_to_list(asf_fullx.model,X_full), axis=0)/68700]

In [39]:
['{:11.2f}'.format(r) for r in kyle1]

We can see that projecting the reference point to the entire data takes way too long and also that our reference was quite optimistic for the first objective.

## Comparing scalarizations

Now we have ASF, STOM ja GUESS scalarizations available. Lets compare them

In [14]:
ref_test = np.array((2,3,4,2))

In [15]:
asf = ASF(ide, nad, ref_test, x, scalarization='ASF')
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [16]:
stom = ASF(ide, nad, ref_test, x, scalarization='STOM')
opt.solve(stom.model)
print(np.sum(values_to_list(stom.model, X), axis=0))

In [17]:
guess = ASF(ide, nad, ref_test, x, scalarization='GUESS')
opt.solve(guess.model)
print(np.sum(values_to_list(guess.model, X), axis=0))

There truly is differences between results, so it seems that scalarization functions are working properly (?)

In [18]:
    from time import time
    from datetime import timedelta
    import logging
    import interactiveBoreal as ib

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    start = time()
    logger.info('Started')
    logger.info('Initializing...')
    kehys = ib.ReferenceFrame()
    logger.info('Initialized. Time since start {}'.
                format(timedelta(seconds=int(time()-start))))
    nclust = 600
    seedn = 6
    '''
    logger.info('Clustering...')
    import simplejson as json
    with open('clusterings/new_300.json', 'r') as file:
        clustering = json.load(file)
    kehys.xtoc = np.array(clustering['5']['xtoc'])
    kehys.weights = np.array([sum(kehys.xtoc == i)
                              for i in range(nclust)
                              if sum(kehys.xtoc == i) > 0])
    
    kehys.cluster(nclust=nclust, seedn=seedn)
    logger.info('Clustered. Time since start {}'.
                format(timedelta(seconds=int(time()-start))))
    '''
    kyle1 = np.array((1858.0, 58.69, 2.651, 0.27))
    init_ref = kyle1 * 68700
    # init_ref = np.array((0, 0, kehys.ideal[2], 0))
    ref = kehys.normalize_ref(init_ref)

    logger.info('Using ideal: {} and nadir: {}'.
                format(kehys.ideal, kehys.nadir))
    logger.info('Reference point: {}.'.format(init_ref))
    logger.info('Solving...')

    '''
    data = kehys.centers
    nvar = len(kehys.x_norm)
    weights = kehys.weights/nvar
    '''
    data = kehys.x_norm_stack
    nvar = len(kehys.x_norm)
    weights = np.ones(nvar)/nvar

    ''' Because everything is scaled, scale these too'''
    ideal = kehys.normalize_ref(kehys.ideal)
    nadir = kehys.normalize_ref(kehys.nadir)

    solver_name = 'cplex'

    asf = ASF(ideal, nadir, ref, data, weights=weights, nvar=nvar,
              scalarization='asf')

    asf_solver = ib.Solver(asf.model, solver=solver_name)
    asf_solver.solve()
    logger.info('Solved')

In [19]:
asf_values = kehys.values(kehys.x_stack, weights, model=asf.model)
asf_values

In [20]:
asf_values/68700

In [21]:
init_ref

In [22]:
kehys.ideal, init_ref, kehys.nadir

In [23]:
ideal, ref, nadir
